In [1]:
#1 Mounting the Google Drive

from google.colab import drive
drive.mount('/content/drive')
data_root='/content/drive/My Drive/ChatBot' #Please upload the files in your drive and change the path to it accordingly.

Mounted at /content/drive


In [2]:
#2 Importing Relevant Libraries

import json
import string
import random

import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [5]:
#3 Loading the Dataset: intents.json

data_file = open('/content/intents.json').read()
data = json.loads(data_file)

data

{'intents': [{'tag': 'hello',
   'patterns': ['Hello', 'Hi there', 'Good morning', "What's up"],
   'responses': ['Hey!', 'Hello', 'Hi!', 'Good morning!'],
   'context': ''},
  {'tag': 'noanswer',
   'patterns': [],
   'responses': ["Sorry, can't understand you",
    'Please give me more info',
    'Not sure I understand'],
   'context': ['']},
  {'tag': 'job',
   'patterns': ['What is your job', 'What is your work'],
   'responses': ['My job is to make you feel like everything is okay.',
    'I work to serve you as well as possible'],
   'context': ''},
  {'tag': 'age',
   'patterns': ['What is your age', 'How old are you', 'When were you born'],
   'responses': ['I was born in 2021'],
   'context': ''},
  {'tag': 'feeling',
   'patterns': ['How are you today', 'How are you'],
   'responses': ['I am feeling good, you?',
    'Very good and you?',
    "Actually, I'm okay and you?"],
   'context': ''},
  {'tag': 'good',
   'patterns': ['I am good too',
    'I feel fine',
    'Good !',
  

In [6]:
#4 Extracting data_X(features) and data_Y(Target)

words = [] #For Bow model/ vocabulary for patterns
classes = [] #For Bow  model/ vocabulary for tags
data_X = [] #For storing each pattern
data_y = [] #For storing tag corresponding to each pattern in data_X
# Iterating over all the intents

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens = nltk.word_tokenize(pattern) # tokenize each pattern
        words.extend(tokens) #and append tokens to words
        data_X.append(pattern) #appending pattern to data_X
        data_y.append(intent["tag"]) ,# appending the associated tag to each pattern

    # adding the tag to the classes if it's not there already
    if intent["tag"] not in classes:
        classes.append(intent["tag"])

# initializing lemmatizer to get stem of words
lemmatizer = WordNetLemmatizer()

# lemmatize all the words in the vocab and convert them to lowercase
# if the words don't appear in punctuation
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
# sorting the vocab and classes in alphabetical order and taking the # set to ensure no duplicates occur
words = sorted(set(words))
classes = sorted(set(classes))

In [7]:
# 5 Text to Numbers
training = []
out_empty = [0] * len(classes)
# creating the bag of words model
for idx, doc in enumerate(data_X):
    bow = []
    text = lemmatizer.lemmatize(doc.lower())
    for word in words:
        bow.append(1) if word in text else bow.append(0)
    # mark the index of class that the current pattern is associated
    # to
    output_row = list(out_empty)
    output_row[classes.index(data_y[idx])] = 1
    # add the one hot encoded BoW and associated classes to training
    training.append([bow, output_row])
# shuffle the data and convert it to an array
random.shuffle(training)
training = np.array(training, dtype=object)
# split the features and target labels
train_X = np.array(list(training[:, 0]))
train_Y = np.array(list(training[:, 1]))

In [10]:
#6 The Neural Network Model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_X[0]),), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_Y[0]), activation = "softmax"))
adam = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=["accuracy"])
print(model.summary())
model.fit(x=train_X, y=train_Y, epochs=150, verbose=1)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 128)               7552      
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                                                 
 dense_7 (Dense)             (None, 64)                8256      
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_8 (Dense)             (None, 16)                1040      
                                                                 
Total params: 16848 (65.81 KB)
Trainable params: 16848 (65.81 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/150
2

In [12]:
#7 Preprocessing the Input

def clean_text(text):
  tokens = nltk.word_tokenize(text)
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  return tokens

def bag_of_words(text, vocab):
  tokens = clean_text(text)
  bow = [0] * len(vocab)
  for w in tokens:
    for idx, word in enumerate(vocab):
      if word == w:
        bow[idx] = 1
  return np.array(bow)

def pred_class(text, vocab, labels):
  bow = bag_of_words(text, vocab)
  result = model.predict(np.array([bow]))[0] #Extracting probabilities
  thresh = 0.5
  y_pred = [[indx, res] for indx, res in enumerate(result) if res > thresh]
  y_pred.sort(key=lambda x: x[1], reverse=True) #Sorting by values of probability in decreasing order
  return_list = []
  for r in y_pred:
    return_list.append(labels[r[0]]) #Contains labels(tags) for highest probability
  return return_list

def get_response(intents_list, intents_json):
  if len(intents_list) == 0:
    result = "Sorry! I don't understand."
  else:
    tag = intents_list[0]
    list_of_intents = intents_json["intents"]
    for i in list_of_intents:
      if i["tag"] == tag:
        result = random.choice(i["responses"])
        break
  return result

In [13]:
# Interacting with the chatbot
print("Press 0 if you don't want to chat with our ChatBot.")
while True:
    message = input("")
    if message == "0":
      break
    intents = pred_class(message, words, classes)
    result = get_response(intents, data)
    print(result)

Press 0 if you don't want to chat with our ChatBot.
Hi
1/1 [==============================] - 0s 86ms/step
That is perfect!
How are you?
1/1 [==============================] - 0s 20ms/step
No, I am a women
I know that whats your name ?
1/1 [==============================] - 0s 25ms/step
Sorry! I don't understand.
your name 
1/1 [==============================] - 0s 24ms/step
Sorry! I don't understand.
Hello
1/1 [==============================] - 0s 22ms/step
That is perfect!
fine
1/1 [==============================] - 0s 20ms/step
So, everything's okay!
yes
1/1 [==============================] - 0s 20ms/step
That is perfect!
good morning
1/1 [==============================] - 0s 22ms/step
Hello
jai mata di
1/1 [==============================] - 0s 33ms/step
That is perfect!
Eid mubarak
1/1 [==============================] - 0s 20ms/step
So, everything's okay!
yes
1/1 [==============================] - 0s 19ms/step
So, everything's okay!
0
